<a href="https://colab.research.google.com/github/sayarghoshroy/proc_s2orc/blob/main/create_rel_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
path = 'drive/My Drive/s2orc/'

read_file = 'pdf_parses_1.jsonl'
rel_work_file = 'out/output_b.jsonl'

MAX_PROC = int(1e6)

In [2]:
# Read a jsonl and create a jsonl
import re
import ast
import json
import tqdm

def normalize(name):
  name = name.lower().strip()
  re.sub('\s+', ' ', name)
  return name

def check_rel_work(body_text):
  for blob in body_text:
    if normalize(blob['section']) == 'related work':
      return True, blob
  return False, None

valid_lines = 0

with open(path + read_file, 'r+') as f:
  for counter in tqdm.tqdm(range(MAX_PROC)):
    line = f.readline();
    if not line:
        break
    paper_dict = json.loads(line)
    if paper_dict['body_text'] == []:
      continue

    check, blob = check_rel_work(paper_dict['body_text'])
    if check:
      valid_lines += 1
      unit = {}
      unit['paper_id'] = paper_dict['paper_id']
      unit['related_work'] = blob
      with open(path + rel_work_file, 'a+') as o:
        o.write(str(json.dumps(ast.literal_eval(str(unit)))))
        o.write('\n')

print('\n\n' + 'Number of entries processed: ' + str(counter))
print('Number of entries with the Related Works sections: ' + str(valid_lines))

 31%|███       | 310316/1000000 [02:19<05:10, 2223.41it/s]



Number of entries processed: 310316
Number of entries with the Related Works sections: 1841


In [3]:
# That's it